### Data Prep

In [ ]:
from datasets import load_dataset
ds = load_dataset("microsoft/cats_vs_dogs")

In [ ]:
import numpy as np

In [ ]:
from PIL import Image, ImageOps


In [ ]:
def preprocessing(example):
    import numpy as np
    size = (64,64)
    img = example['image']
    img = img.convert('RGB')
    img_resized = img.resize(size)
    img_processed = np.array(img_resized, dtype=np.float32)/255
    img_flatten = img_processed.flatten()
    example['image'] = img_flatten
    return example

ds_resized = ds['train'].map(preprocessing, num_proc=4)


In [ ]:
ds_resized.shape

In [ ]:
ds_resized['image']

In [ ]:
vector_img = np.array(ds_resized['image'][:15000])

In [ ]:
vector_img = vector_img.squeeze()

In [ ]:
vector_img.shape

In [ ]:
vector_labels = np.array(ds['train']['labels'][:15000],dtype = bool)

In [ ]:
vector_labels.shape

In [ ]:
# def split_function(database, x):
#     size_train = round(ds_flatten.shape[0]*x)
#     split_train = database[:size_train]
#     split_test = database[size_train:]
#     return split_train, split_test

# (split_train, split_test) = split_function(ds_flatten,x = 0.8)
# split_test



### NN 
Vectorization


In [ ]:
import numpy as np
logistic_regression = {}

def log_reg(z):
    return 1/(1+np.exp(-z))

def d_log_reg(a):
    return a * (1 - a)
logistic_regression['activation'] = log_reg
logistic_regression['derivate'] = d_log_reg

ReLinearUnit = {}
def relu(z):
    return np.maximum(0,z)

def d_relu(z):
    return (z > 0).astype(float)

ReLinearUnit['activation'] = relu
ReLinearUnit['derivate'] = d_relu

class NN():
    def __init__(self,L,nx,learning_rate,activation_funcs): #only hidden layers
        self.L = L #number of hidden layers
        self.nx = nx #number of units per layer
        self.learning_rate = learning_rate
        self.activation_funcs = activation_funcs
        self.weights = [None] * self.L
        print(self.weights)
        self.bias = [None] * self.L
        self.dW = [None] * self.L
        self.Z = [None] * self.L
        self.A = [None] * self.L
        self.dZ = [None] * self.L
        self.dA = [None] * self.L
        self.db = [None] * self.L


    def random_weights(self,n0):
        self.weights[0] = np.random.randn(self.nx[0],n0)* np.sqrt(2 / self.nx[0])
        #self.weights[0] = np.random.randn(n_in, n_out) * np.sqrt(2 / n_in)

        self.bias[0] = np.random.randn(self.nx[0],1)* np.sqrt(2 / self.nx[0])
        print('w shape, layer :',0,':',np.shape(self.weights[0]))

        for layer in range(1,self.L):
            self.weights[layer] = np.random.randn(self.nx[layer],self.nx[layer-1])* np.sqrt(2 / self.nx[layer-1])
            self.bias[layer] = np.random.randn(self.nx[layer],1)* np.sqrt(2 / self.nx[layer-1])
            print('w shape, layer :',layer,':',np.shape(self.weights[layer]))
            print(self.weights[layer],self.bias[layer])
        print('random weights initialized')
        print('w',self.weights[self.L-1],'b',self.bias[self.L-1])
        
    
    def forward_pass(self,X,Y):
        self.m = X.shape[1]
        print(self.m)
        self.Z[0] = np.dot(self.weights[0],X.T)+self.bias[0] # T car 
        print('ZO',self.Z[0])
        self.A[0] = self.activation_funcs[0]['activation'](self.Z[0])
        for layer in range(1,self.L):
            self.Z[layer] = np.dot(self.weights[layer],self.A[layer-1])+self.bias[layer]
            self.A[layer] = self.activation_funcs[layer]['activation'](self.Z[layer])
            print('A layer ',layer, ':', self.A[layer])
        #cost computation
        A = self.A[self.L-1]
        print('A0 shape:', self.A[0].shape)
        print('A shape : ',A.shape)
        print('Y shape : ', Y.shape)
        cost = -1/self.m*np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))
        return cost
    
    def backward_pass(self,X,Y):
        dL = 1
        L = self.L
        self.dA[L-1] = -1/self.m*((Y/self.A[L-1])-(1-Y)/(1-self.A[L-1])) 
        self.dZ[L-1] = self.dA[L-1]*self.activation_funcs[L-1]['derivate'](self.A[L-1]) #(n[l-1], m) da(n[l-1],m) a (n[l-1],m)
        self.dW[L-1] = 1/self.m*np.dot(self.dZ[L-1],self.A[L-2].T) # = (n[l-1],m)(n[l-2],m)
        print('dW[L-1] :',self.dW[L-1])
        self.db[L-1] = np.mean(self.dZ[L-1], axis= 1,keepdims=True)
        for layer in range(self.L-2,0,-1):
            self.dA[layer] =  np.dot(self.weights[layer+1].T,self.dZ[layer+1]) 
            self.dZ[layer] = self.dA[layer]*self.activation_funcs[layer]['derivate'](self.A[layer]) 
            self.dW[layer] = 1/self.m*np.dot(self.dZ[layer],self.A[layer-1].T) #(n[l],n[l-1]) = (n[l],m)(m,n[l-1]) => où passe les m et comment faire pour avoir une update moyenne
            print('dW',layer,self.dW[layer])
            self.db[layer] = np.mean(self.dZ[layer], axis= 1,keepdims=True) # (n[l],1) => avec la moyenne 
        self.dA[0] =  np.dot(self.weights[1].T,self.dZ[1]) 
        self.dZ[0] = self.dA[0]*self.activation_funcs[0]['derivate'](self.A[0]) 
        self.dW[0] = 1/self.m*np.dot(self.dZ[0],X) #(10,12288)
        print('dW[0] :',self.dW[L-1])

        self.db[0] = np.mean(self.dZ[0], axis= 1,keepdims=True) # (n[l],1) => avec la moyenne 

        #update weights
        for layer in range(self.L-1,-1,-1):
            self.weights[layer] = self.weights[layer] -self.learning_rate*self.dW[layer]
            self.bias[layer] = self.bias[layer] - self.learning_rate*self.db[layer]
        


    
    


In [ ]:
quick_test = NN(L = 9,
                nx = np.array([10,9,8,7,8,9,10,3,1]),
                learning_rate=0.001,
                activation_funcs=[ReLinearUnit,ReLinearUnit,ReLinearUnit,ReLinearUnit,ReLinearUnit,ReLinearUnit,ReLinearUnit,ReLinearUnit,logistic_regression])

In [ ]:
quick_test.random_weights(n0 = np.shape(vector_img)[1])

In [ ]:
last_cost =[]
for i in range(100):
    quick_test.forward_pass(vector_img,vector_labels)
    quick_test.backward_pass(vector_img,vector_labels)
    last_cost.append(quick_test.forward_pass(vector_img,vector_labels))
print(last_cost)

In [ ]:
import matplotlib.pyplot as plt

x = [i for i in range(100)]
print(last_cost)
plt.plot(x, last_cost)
plt.ylim(min(last_cost), max(last_cost))
plt.ylabel("Cost")
plt.title("Évolution de la cost")
plt.grid(True)
plt.show()
